In [3]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import fitz  # PyMuPDF
import cv2
import pytesseract
import numpy as np
import os

def isolate_dark_shades(image_path, dark_shades_count=2):
    """
    Изолирует самые темные оттенки на изображении, заменяя остальные на белый цвет.

    :param image_path: Путь к исходному изображению.
    :param dark_shades_count: Количество темных оттенков для сохранения.
    :return: Изображение только с темными оттенками и белым фоном.
    """
    # Загрузка и подготовка изображения
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_flat = img.reshape((-1, 3))

    # Применение k-средних для нахождения основных цветов
    kmeans = KMeans(n_clusters=5, n_init=10)
    kmeans.fit(img_flat)
    dominant_colors = kmeans.cluster_centers_

    # Сортировка цветов по их яркости
    colors_brightness = np.sum(dominant_colors, axis=1)
    sorted_indices = np.argsort(colors_brightness)

    # Выбор самых темных оттенков
    dark_colors = sorted_indices[:dark_shades_count]

    # Замена пикселей, не относящихся к темным оттенкам, на белый
    for i in range(len(img_flat)):
        if kmeans.labels_[i] not in dark_colors:
            img_flat[i] = [255, 255, 255]

    return img_flat.reshape(img.shape)

    
def remove_spots(image, kernel_size=(3, 3), area_threshold=100):
    """
    Удаляет мелкие изолированные элементы (кляксы) из изображения.

    :param image: Входное изображение для обработки.
    :param kernel_size: Размер ядра для морфологических операций.
    :param area_threshold: Пороговое значение площади для удаления мелких контуров.
    :return: Обработанное изображение.
    """
    # Морфологическая эрозия и дилатация
    kernel = np.ones(kernel_size, np.uint8)
    erosion = cv2.erode(image, kernel, iterations=1)
    dilation = cv2.dilate(erosion, kernel, iterations=1)

    # Найти контуры на изображении
    contours, _ = cv2.findContours(dilation, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Создать маску для удаления мелких контуров
    mask = np.ones_like(dilation) * 255
    for contour in contours:
        area = cv2.contourArea(contour)
        if area < area_threshold:
            cv2.drawContours(mask, [contour], -1, 0, -1)

    # Применить маску к изображению
    return cv2.bitwise_and(dilation, dilation, mask=mask)


In [4]:
# Путь к PDF файлу и путь к Tesseract
pdf_path = 'Kogan_1.pdf'
tesseract_path = 'C:/Program Files/Tesseract-OCR/tesseract.exe'
pytesseract.pytesseract.tesseract_cmd = tesseract_path

# Папка для сохранения изображений
images_folder = 'extracted_images'
os.makedirs(images_folder, exist_ok=True)

print("Начало обработки PDF файла.")

# Открыть PDF файл
doc = fitz.open(pdf_path)

# Файл для сохранения извлеченного текста
extracted_text_path = 'extracted_text.txt'

# Открыть файл для записи извлеченного текста
with open(extracted_text_path, 'w', encoding='utf-8') as text_file:

    # Перебрать страницы и извлечь изображения
    for page_num in range(129, 135):  # Страницы с 130 по 135
        print(f"Обработка страницы {page_num + 1}...")
        page = doc.load_page(page_num)
        pix = page.get_pixmap(matrix=fitz.Matrix(5, 5))
        img_data = pix.tobytes("png")
        img = cv2.imdecode(np.frombuffer(img_data, np.uint8), 1)

        # Сохранение оригинального изображения
        original_img_path = os.path.join(images_folder, f'page_{page_num + 1}_original.png')
        cv2.imwrite(original_img_path, img)       

        # Обработка сохраненного изображения
        img = isolate_dark_shades(original_img_path, dark_shades_count=2)
        if img is not None:
            cv2.imwrite(original_img_path, img)       

        # Адаптивная пороговая обработка и удаление "клякс"
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)
        processed_img = remove_spots(thresh, kernel_size=(5, 5), area_threshold=100)        

        # Сохранить обработанное изображение и анализировать цветовую палитру
        processed_img_path = os.path.join(images_folder, f'page_{page_num + 1}_processed.png')
        cv2.imwrite(processed_img_path, thresh)        

        # OCR и сохранение текста
        custom_config = r'--oem 3 --psm 6'
        text = pytesseract.image_to_string(thresh, config=custom_config, lang='rus+eng')
        text_file.write(f'--- Страница {page_num + 1} ---\n{text}\n')

# Закрыть PDF файл
doc.close()

print("Обработка PDF файла завершена.")


Начало обработки PDF файла.
Обработка страницы 130...
Обработка страницы 131...
Обработка страницы 132...
Обработка страницы 133...
Обработка страницы 134...
Обработка страницы 135...
Обработка PDF файла завершена.
